In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [242]:
df=pd.read_csv('../BCI/data/bci/first.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,61,62,63,64,65,66,67,68,69,70
0,0,3.618,0.687,0.824,0.542,0.394,4.476,2.162,0.369,0.583,...,1.800,0.341,0.888,0.401,0.429,0.264,0.068,0.114,0.046,Esquerda
1,1,4.341,1.421,0.931,0.643,0.385,4.857,2.668,0.470,0.713,...,2.734,0.543,0.943,0.445,0.507,0.321,0.073,0.118,0.053,Esquerda
2,2,5.599,2.591,1.225,0.785,0.398,6.259,3.332,0.667,0.882,...,3.870,0.933,1.009,0.484,0.570,0.391,0.080,0.117,0.064,Esquerda
3,3,7.740,4.040,1.711,0.948,0.437,9.387,4.137,0.904,1.085,...,4.991,1.453,1.077,0.511,0.600,0.470,0.089,0.114,0.077,Esquerda
4,4,10.818,5.428,2.283,1.095,0.494,14.438,4.939,1.112,1.290,...,5.849,1.990,1.126,0.523,0.598,0.550,0.098,0.110,0.089,Esquerda


In [243]:
df=df.drop(columns=['Unnamed: 0'])
df.shape

(296, 71)

In [244]:
df=df.dropna(axis=0)
df.shape

(296, 71)

In [245]:
theta=[str(a*5) for a in range(0,14)]
alpha=[str(1+a*5) for a in range(0,14)]
low_beta=[str(2+a*5) for a in range(0,14)]
high_beta=[str(3+a*5) for a in range(0,14)]
gamma=[str(4+a*5) for a in range(0,14)]

In [246]:
df_theta=df.drop(columns=[str(a) for a in range(0,70) if str(a) not in theta])
df_alpha=df.drop(columns=[str(a) for a in range(0,70) if str(a) not in alpha])
df_low_beta=df.drop(columns=[str(a) for a in range(0,70) if str(a) not in low_beta])
df_high_beta=df.drop(columns=[str(a) for a in range(0,70) if str(a) not in high_beta])
df_gamma=df.drop(columns=[str(a) for a in range(0,70) if str(a) not in gamma])

In [328]:
X=df_gamma.iloc[:,:-1].values
y=df_gamma.iloc[:,-1].values

In [288]:
from sklearn.ensemble import RandomForestClassifier
pipeline=RandomForestClassifier()

In [249]:
from sklearn.model_selection import GridSearchCV
params = {  
      'nthread':[4,5,6], #when use hyperthread, xgboost may become slower
      'objective':['binary:logistic'],
      'silent': [1,2,3],
      'subsample': [0.8,0.9],
      'n_estimators': [10,100,1000,10000], #number of trees, change it to 1000 for better results
      'missing':[-999],
#       'seed': [1337],
      'early_stopping':[True],
}


In [252]:
from xgboost import XGBClassifier
grid=GridSearchCV(estimator=XGBClassifier(),
                 param_grid=params,
                 cv=10,
                 n_jobs=-1)

In [253]:
grid.fit(X,y)

GridSearchCV(cv=10, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'early_stopping': [True], 'missing': [-999],
                         'n_estimators': [10, 100, 1000, 10000],
                         'nthread': [4, 5, 

In [335]:
pipeline=grid.best_estimator_
pipeline.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, early_stopping=True,
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=-999, n_estimators=100, n_jobs=1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)

In [336]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred=cross_val_predict(pipeline,X,y,cv=10)
print(confusion_matrix(y,y_pred))

[[57 25 16]
 [36 61  2]
 [13  8 78]]


In [337]:
#Outras métricas relevantes
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

     Direita       0.54      0.58      0.56        98
    Esquerda       0.65      0.62      0.63        99
      Neutro       0.81      0.79      0.80        99

    accuracy                           0.66       296
   macro avg       0.67      0.66      0.66       296
weighted avg       0.67      0.66      0.66       296



In [338]:
from sklearn.feature_selection import SelectFromModel
model=SelectFromModel(pipeline, prefit=True)

In [339]:
X_new = model.transform(X)

In [340]:
y_pred=cross_val_predict(pipeline,X_new,y,cv=10)
print(confusion_matrix(y,y_pred))

[[69 15 14]
 [25 68  6]
 [17  2 80]]


In [341]:
#Outras métricas relevantes
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

     Direita       0.62      0.70      0.66        98
    Esquerda       0.80      0.69      0.74        99
      Neutro       0.80      0.81      0.80        99

    accuracy                           0.73       296
   macro avg       0.74      0.73      0.73       296
weighted avg       0.74      0.73      0.73       296

